In [1]:
from data_gen import generate_data
from train import train
import numpy as np
import pandas as pd

In [2]:
# default args
datagen_args={}
datagen_args['T'] = 1000
datagen_args['corr'] = 1.0
datagen_args['A'] = 2
datagen_args['N'] = 10
datagen_args['M'] = 1
datagen_args['K'] = 8
datagen_args['num_seeds'] = 2
datagen_args['action_selection_method'] = 'greedy'
datagen_args['ensemble'] = 'sum'
datagen_args['ground_model_name'] = 'bitpop'
datagen_args['output'] = 'output/'

train_args = {}
train_args['model_name']='stomp'
train_args['P']=1e6
train_args['M']=1
train_args['L']=100
train_args['n_hidden_layers']=2
train_args['n_features']=2
train_args['num_codebooks']=10
train_args['enc2dec_ratio']=1
train_args['epochs']=20
train_args['learning_rate']=5e-5
train_args['batch_size']=8
train_args['outdir']='output/'
train_args['data_dir']=''
train_args['seed']=0
train_args['data_seed']=0
train_args['use_lr_scheduler']=False
train_args['step_LR']=30
train_args['gamma']=0.1
train_args['checkpoint_interval']=100
train_args['wandb_entity_name']=None
train_args['wandb_group_name']=None
train_args['wandb_job_type_name']=None

In [3]:
# variable parameters
corrvec=[0,1] # [0, 0.5, 1.0]   # agent pairwise action correlation
Nvec=[10,100] # [1e1,1e2,1e3,1e4]   # number of agents
single_agent_capacity = 256*100
Pvec=[Nvec[0]*single_agent_capacity,Nvec[1]*single_agent_capacity]  # number of parameters

# datagen vars
K=8             # state space dimension
M_sys=1         # number of agent groups
T=int(1e4)      # number of samples to learn from
data_seed = 0   # seed of data generation

# train vars
M_train=1       # assumed number of agent groups
epochs=100

datagen_args['K']=K
datagen_args['M']=M_sys
datagen_args['T']=T
datagen_args['ground_model_name'] = 'bitpop'

train_args['model_name']='single'
train_args['M']=M_train
train_args['data_seed']=data_seed
train_args['epochs']=epochs

In [4]:
# generate data from bitpop
hashtype='bitpop_data'
hash_data_list=[]
for corr in corrvec:
    datagen_args['corr']=corr
    for N in Nvec:
        datagen_args['N']=N
        bitpop_data_hash=generate_data(datagen_args.copy())
        hash_data_list.append((corr,N,-1,hashtype,bitpop_data_hash))
df=pd.DataFrame(hash_data_list,columns=['corr','N','P','hashtype','hash'])

bitpop
running seed 0 of 2
running seed 1 of 2
saving data_d1bac3a730
bitpop
running seed 0 of 2
running seed 1 of 2
saving data_4cace0bd77
bitpop
running seed 0 of 2
running seed 1 of 2
saving data_104df71b36
bitpop
running seed 0 of 2
running seed 1 of 2
saving data_ab133d9f57


In [5]:
# train simple
hashtype='train_simple'
hash_data_list=[]
for corr in corrvec:
    for nit,N in enumerate(Nvec):
        train_args['data_dir']='data_' + df.loc[
            (df['corr']==corr) & (df['N']==N) & (df['hashtype']=='bitpop_data'),'hash'].values[0]
        train_args['P']=Pvec[nit]
        print(type(train_args['P']))
        train_simple_hash=train(train_args.copy())
        hash_data_list.append((corr,N,Pvec[nit],hashtype,train_simple_hash))
dftmp=pd.DataFrame(hash_data_list,columns=['corr','N','P','hashtype','hash'])
df=pd.concat((df,dftmp))

<class 'int'>
Using cpu device
seed 0 training of single model with modelsize 256000 for 100 epochs using batchsize 8 and LR 5e-05
wandb run name: 7eab1caa5f_20240524-004952
using data:output/data_d1bac3a730/data.h5
{'file_attrs': {'A': 2, 'K': 8, 'M': 1, 'N': 10, 'T': 10000, 'action_selection_method': 'greedy', 'corr': 0, 'ensemble': 'sum', 'ground_model_name': 'bitpop', 'hash': 'd1bac3a730', 'num_seeds': 2, 'output': 'output/', 'timestamp': '20240524_004948'}}
state_dim: (10000, 8)
256000
hidden_dim=110
number of parameters: 256320
gap between P and num_parameters:  -320
pre training loss: 0.08657589222431183, acc: 0.5168
Epoch 1, loss: 0.080097453, acc: 0.61226
Epoch 2, loss: 0.079136306, acc: 0.61265
Epoch 3, loss: 0.079112449, acc: 0.61266
Epoch 4, loss: 0.079122773, acc: 0.61263
Epoch 5, loss: 0.079106439, acc: 0.61245
Epoch 6, loss: 0.079114488, acc: 0.61259
Epoch 7, loss: 0.079110655, acc: 0.61313
Epoch 8, loss: 0.079111715, acc: 0.61167
Epoch 9, loss: 0.07909729, acc: 0.61232


Epoch 63, loss: 0.079529977, acc: 0.613753
Epoch 64, loss: 0.079524267, acc: 0.613707
Epoch 65, loss: 0.079522153, acc: 0.613796
Epoch 66, loss: 0.079525317, acc: 0.613811
Epoch 67, loss: 0.079527138, acc: 0.613831
Epoch 68, loss: 0.079525325, acc: 0.613707
Epoch 69, loss: 0.079525699, acc: 0.613811
Epoch 70, loss: 0.079526095, acc: 0.613827
Epoch 71, loss: 0.079528754, acc: 0.613804
Epoch 72, loss: 0.079526405, acc: 0.613781
Epoch 73, loss: 0.079525793, acc: 0.613809
Epoch 74, loss: 0.079526894, acc: 0.6137
Epoch 75, loss: 0.079524112, acc: 0.613875
Epoch 76, loss: 0.079524824, acc: 0.613759
Epoch 77, loss: 0.079528448, acc: 0.613629
Epoch 78, loss: 0.079523756, acc: 0.613717
Epoch 79, loss: 0.079525323, acc: 0.613747
Epoch 80, loss: 0.079526001, acc: 0.613683
Epoch 81, loss: 0.079527118, acc: 0.613711
Epoch 82, loss: 0.079526941, acc: 0.613693
Epoch 83, loss: 0.079523187, acc: 0.613785
Epoch 84, loss: 0.079524541, acc: 0.613702
Epoch 85, loss: 0.079528241, acc: 0.613775
Epoch 86, los

Epoch 27, loss: 0.083620149, acc: 0.595
Epoch 28, loss: 0.083599263, acc: 0.595
Epoch 29, loss: 0.083614097, acc: 0.595
Epoch 30, loss: 0.083614538, acc: 0.595
Epoch 31, loss: 0.083617145, acc: 0.595
Epoch 32, loss: 0.083612831, acc: 0.595
Epoch 33, loss: 0.083609752, acc: 0.595
Epoch 34, loss: 0.083609127, acc: 0.595
Epoch 35, loss: 0.083596817, acc: 0.595
Epoch 36, loss: 0.083624121, acc: 0.595
Epoch 37, loss: 0.08359702, acc: 0.595
Epoch 38, loss: 0.083600846, acc: 0.595
Epoch 39, loss: 0.083600626, acc: 0.595
Epoch 40, loss: 0.083611497, acc: 0.595
Epoch 41, loss: 0.083617362, acc: 0.595
Epoch 42, loss: 0.083608773, acc: 0.595
Epoch 43, loss: 0.083612049, acc: 0.595
Epoch 44, loss: 0.08360888, acc: 0.595
Epoch 45, loss: 0.083601897, acc: 0.595
Epoch 46, loss: 0.083608047, acc: 0.595
Epoch 47, loss: 0.083605326, acc: 0.595
Epoch 48, loss: 0.083590352, acc: 0.595
Epoch 49, loss: 0.083614824, acc: 0.595
Epoch 50, loss: 0.083601117, acc: 0.595
Epoch 51, loss: 0.083619516, acc: 0.595
Ep

In [6]:
# generate data from trained simple
hashtype = 'simple_data'
hash_data_list=[]
for corr in corrvec:
    for nit,N in enumerate(Nvec):
        data_hash=df.loc[
            (df['corr']==corr) & (df['N']==N) & (df['hashtype']=='bitpop_data'),'hash'].values[0]
        train_hash=df.loc[
            (df['corr']==corr) & (df['N']==N) & (df['hashtype']=='train_simple'),'hash'].values[0]
        datagen_args['ground_model_name'] = data_hash+'/'+train_hash
        simple_data_hash=generate_data(datagen_args.copy())
        hash_data_list.append((corr,N,Pvec[nit],hashtype,simple_data_hash))
dftmp=pd.DataFrame(hash_data_list,columns=['corr','N','P','hashtype','hash'])
df=pd.concat((df,dftmp))

d1bac3a730/7eab1caa5f
data_settings:
{'A': 2, 'K': 8, 'M': 1, 'N': 10, 'T': 10000, 'action_selection_method': 'greedy', 'corr': 0, 'ensemble': 'sum', 'ground_model_name': 'bitpop', 'hash': 'd1bac3a730', 'num_seeds': 2, 'output': 'output/', 'timestamp': '20240524_004948'}
orig_training_args:
{'L': 100, 'M': 1, 'P': 256000, 'batch_size': 8, 'checkpoint_interval': 100, 'data_dir': 'data_d1bac3a730', 'data_seed': 0, 'enc2dec_ratio': 1, 'epochs': 100, 'gamma': 0.1, 'hidden_dim': 110, 'learning_rate': 5e-05, 'model_name': 'single', 'n_features': 2, 'n_hidden_layers': 2, 'num_codebooks': 10, 'outdir': 'output/', 'seed': 0, 'step_LR': 30, 'use_lr_scheduler': False, 'wandb_entity_name': None, 'wandb_group_name': None, 'wandb_job_type_name': None}
parameters overwritten from single model at bitpop
running seed 0 of 2
running seed 1 of 2
saving data_ec6e64862c
4cace0bd77/8280dc1fe8
data_settings:
{'A': 2, 'K': 8, 'M': 1, 'N': 100, 'T': 10000, 'action_selection_method': 'greedy', 'corr': 0, 'ensem

In [7]:
# store
data_store={}
data_store['datagen_args']=datagen_args
data_store['train_args']=train_args
data_store['hashes']=df
data_filename = f"hashlist_K_{K}_Msys_{M_sys}_T_{T}_Mtrain_{M_train}_Ep_{epochs}_dataseed_{data_seed}"
np.save(data_filename+".npy",data_store)
df.to_csv(data_filename, index=False)

In [8]:
df

,corr,N,P,hashtype,hash
0,0,10,-1,bitpop_data,d1bac3a730
1,0,100,-1,bitpop_data,4cace0bd77
2,1,10,-1,bitpop_data,104df71b36
3,1,100,-1,bitpop_data,ab133d9f57
0,0,10,256000,train_simple,7eab1caa5f
1,0,100,2560000,train_simple,8280dc1fe8
2,1,10,256000,train_simple,032be1f813
3,1,100,2560000,train_simple,d47de5188e
0,0,10,256000,simple_data,ec6e64862c
1,0,100,2560000,simple_data,16ad0eabc1


In [9]:

# #train match
# train_args['model_name']='match'
# match_train_hashes = []
# for data_hash in simple_data_hashes:
#   train_args['data_dir']='data_'+data_hash
#   match_train_hashes.append(train(train_args.copy()))
# # write_hashes(match_train_hashes,'match_train')
# hashlist_dict['match_train']=match_train_hashes
# print(''.join(['\n']*10))
# np.save(hashlist_filename,hashlist_dict)

# #train mlp
# train_args['model_name']='match'
# match_train_hashes = []
# for data_hash in simple_data_hashes:
#   train_args['data_dir']='data_'+data_hash
#   match_train_hashes.append(train(train_args.copy()))
# # write_hashes(match_train_hashes,'match_train')
# hashlist_dict['match_train']=match_train_hashes
# print(''.join(['\n']*10))
# np.save(hashlist_filename,hashlist_dict)

# #generate data from trained match
# match_data_hashes=[]
# for datahash,trainhash in zip(simple_data_hashes,match_train_hashes):
#   datagen_args['ground_model_name'] = datahash+'/'+trainhash
#   match_data_hashes.append(generate_data(datagen_args.copy()))
# write_hashes(match_data_hashes,'match_data')
# hashlist_dict['match_data']=match_data_hashes
# print(''.join(['\n']*10))

# np.save(hashlist_filename,hashlist_dict)


# def write_hashes(hash_list,hash_name,file_name=hashlist_filename):
#   with open(file_name,'a') as f:
#       f.write(hash_name)
#       for ha in hash_list:
#           f.write(ha)
# write_hashes(bitpop_data_hashes,'bitpop_data')
# write_hashes(simple_train_hashes,'simple_train')
# write_hashes(simple_data_hashes,'single_data')
